In [1]:
import random
import csv
import math
import numpy as np
import time
import codecs

In [2]:
# 1 email có rất nhiều thời gian 
def orderTimestamps(filename):
    # Khởi tạo 1 dictionary timestampsDict để lưu trữ thông tin về các timestamps(về số lần xuất hiện của mỗi cạnh)
    timestampsDict = {}

    # Mở file và tiến hành đọc các byte từ các tệp tin
    with open(filename,'rb') as tsvin:
        # Đầu tiên, codecs.iterdecode(tsvin, 'utf-8') chuyển đổi dữ liệu từ định dạng byte sang chuỗi sử dụng bảng mã UTF-8. Sau đó, csv.reader() 
        # dùng để đọc dữ liệu từ chuỗi này dưới dạng các hàng và cột, với delimiter='\t' chỉ định rằng các giá trị được phân tách bằng tab.
        tsvin = csv.reader(codecs.iterdecode(tsvin, 'utf-8'), delimiter='\t')
        # Bỏ qua dòng tiêu đề (dòng đầu tiên của tệp tin)
        next(tsvin)
        # Duyệt qua từng dòng trong tập dữ liệu 
        for row in tsvin:
            # Lấy giá trị đầu tiên trong hàng (cột thứ 0) và chuyển đổi nó thành số nguyên. Đây là giá trị của srcID.
            srcID = int(row[0])
            # Lấy giá trị thứ hai trong hàng (cột thứ 1) và chuyển đổi nó thành số nguyên. Đây là giá trị của dstID.
            dstID = int(row[1])
            # Lấy giá trị thứ tư trong hàng (cột thứ 3) và chuyển đổi nó thành số nguyên. Đây là giá trị của timestamp.
            timestamp = int(row[3])
            # Tạo một bộ ba (tuple) gồm srcID, dstID, và timestamp, để đại diện cho một cạnh (edge) trong dữ liệu.
            edge = (srcID, dstID, timestamp)
            # Kiểm tra xem edge đã tồn tại trong timestampsDict hay chưa. Nếu đã tồn tại, ta tăng giá trị tương ứng trong timestampsDict lên 1.
            # Nếu edge chưa tồn tại trong timestampsDict, ta thêm edge vào timestampsDict với giá trị ban đầu là 1.
            if edge in timestampsDict:
                timestampsDict[edge] += 1
            else:
                timestampsDict[edge] = 1
    #  sắp xếp timestampsDict theo thứ tự tăng dần của timestamp. 
    #  Kết quả được lưu vào sorted_timestampsDict dưới dạng một danh sách các bộ tứ (tuple) gồm timestamp, srcID, dstID, và số lần xuất hiện.
    sorted_timestampsDict = sorted( (key[2], key[0], key[1], value) for (key, value) in timestampsDict.items() )
    
    
    return sorted_timestampsDict

In [7]:

# Đây là một hàm được sử dụng để tạo ra các cạnh sống (live edges) và thông tin về các nút bị nhiễm trong các đợt bùng phát nhiễm. 
# Hàm này nhận vào các tham số runs, p_initial_infect, p_infect, n, seed, và debug.
# Hàm tạo ra runs đợt bùng phát nhiễm. Mỗi đợt bùng phát được tạo ra như sau:
# Tạo danh sách rỗng cur_infected_ids để lưu trữ các nút bị nhiễm trong đợt bùng phát.
# Chọn ngẫu nhiên các "patient zeros" (nút ban đầu bị nhiễm) và thêm chúng vào cur_infected_ids.
# Lặp qua các timestamps từ tệp tin đã được sắp xếp.
# Kiểm tra các điều kiện để lây nhiễm từ nút nguồn sang nút đích và thêm nút đích vào cur_infected_ids nếu bị nhiễm.
# Cuối cùng, thêm cur_infected_ids vào danh sách lists_infected_nodes và lưu trữ thông tin của đợt bùng phát vào lists_infected_dicts.
# Sau khi chạy xong runs lần, hàm trả về lists_infected_nodes và lists_infected_dicts chứa thông tin về các nút bị nhiễm trong các đợt bùng phát nhiễm.
# Các tham số của hàm cho phép điều chỉnh số lần chạy, xác suất ban đầu để nút bị nhiễm, xác suất lây nhiễm giữa các nút, số lượng nút trong mạng, hạt giống ngẫu nhiên và thông tin gỡ rối.

"""runs:Tham số này xác định số lần chạy để tạo ra các đợt bùng phát nhiễm.

p_initial_infect: Xác suất ban đầu để một nút được chọn là nhiễm.

p_infect: Xác suất lây nhiễm từ một nút nhiễm sang một nút chưa nhiễm trong mỗi lượt truyền.

n: Số lượng nút trong mạng.

seed: Giá trị hạt giống (seed) được sử dụng để khởi tạo bộ sinh số ngẫu nhiên.

debug: Tham số này xác định xem có in thông tin gỡ rối (debug) hay không.

Dòng random.seed(seed) được sử dụng để khởi tạo bộ sinh số ngẫu nhiên với hạt giống được chỉ định.

timestampsOrdered = orderTimestamps('out.dnc-temporalGraph'): Hàm orderTimestamps được gọi để đọc và sắp xếp các timestamps 
từ tệp tin 'out.dnc-temporalGraph'. Kết quả được lưu vào timestampsOrdered.

num_nodes_infect = math.floor(n * p_initial_infect): Tính toán số lượng nút sẽ bị nhiễm ban đầu,
 dựa trên xác suất p_initial_infect và số lượng nút n.

lists_infected_nodes và lists_infected_dicts là các danh sách rỗng được sử dụng để 
lưu trữ thông tin về các nút bị nhiễm trong các đợt bùng phát.

Vòng lặp for run in range(runs): được sử dụng để lặp qua từng lần chạy.

Bên trong vòng lặp, danh sách cur_infected_ids và từ điển cur_infected_dict được tạo ra để lưu trữ các nút 
đã bị nhiễm trong đợt bùng phát hiện tại.

Tiếp theo, trong vòng lặp while, chúng ta chọn ngẫu nhiên các "patient zeros" (các nút ban đầu bị nhiễm).
 Cho đến khi số lượng cur_infected_ids đạt đến num_nodes_infect,
chúng ta chọn ngẫu nhiên một nút từ 1 đến n và thêm nút đó vào cur_infected_ids và cur_infected_dict.

Vòng lặp tiếp theo là vòng lặp for timestamp in timestampsOrdered để lặp qua các timestamps từ tệp tin.

Trong vòng lặp, sourceNode và destNode được lấy từ các giá trị trong timestamp để đại diện cho các nút nguồn và đích trong cạnh.

Tiếp theo, chúng ta kiểm tra các điều kiện để lây nhiễm từ nút nguồn sang nút đích. 
Nếu nút nguồn đã bị nhiễm (sourceNode in cur_infected_ids) và nút đích chưa bị nhiễm (destNode not in cur_infected_ids), 
chúng ta sử dụng xác suất p_infect để xác định xem nút đích có bị nhiễm hay không. 
Nếu nút đích bị nhiễm, chúng ta thêm nó vào cur_infected_ids và cur_infected_dict.

Cuối cùng, danh sách cur_infected_ids và cur_infected_dict của đợt bùng phát hiện tại 
được thêm vào các danh sách chính lists_infected_nodes và lists_infected_dicts để lưu trữ thông tin về các đợt bùng phát.

Sau khi chạy xong runs lần, hàm trả về lists_infected_nodes và lists_infected_dicts 
chứa thông tin về các nút bị nhiễm trong các đợt bùng phát nhiễm."""

def createLiveEdges_combined(runs, p_initial_infect, p_infect, n=2029, seed=0, debug=False):
    random.seed(seed)
    timestampsOrdered = orderTimestamps('out.dnc-temporalGraph')
    
    
    num_nodes_infect = math.floor(n * p_initial_infect)
    lists_infected_nodes = [] # list of lists of infected nodes from every run
    lists_infected_dicts = []  # for steps
     
    
    for run in range(runs): # go thru runs
        if run % 2500 == 0:
            print("Creating Live Edge #" + str(run))
        
        cur_infected_ids = []
        cur_infected_dict = {}
        
        # Mark patient zeros
        while len(cur_infected_ids) < num_nodes_infect:
            nID = random.randint(1,n) 
            if nID not in cur_infected_ids:
                cur_infected_ids.append(nID)
                cur_infected_dict[nID] = 0
        
        step = 0 
        
        for timestamp in timestampsOrdered: # cycle thru the timestamps in the e-mails
            
            sourceNode = timestamp[1]
            destNode = timestamp[2]
            
            # this logic is used to infect neighbors
            if sourceNode in cur_infected_ids: # source node is infected, so we can try to infect
                if destNode not in cur_infected_ids: # destination node isn't infected, so we can try to infect
                    infected = False # flag to track whether infection successfully transmits
                    for email in range(timestamp[3]): # since an edge w/ the same timestamp can occur multiple times
                        if random.uniform(0, 1) < p_infect: # flip a coin to infect neighbor
                            infected = True
                            cur_infected_ids.append(destNode)
                            cur_infected_dict[destNode] = step + email # timestamp of infection
                            break # b/c we do have an infection, so no need to try to keep infecting
            step = step + timestamp[3] # increment the step
            
        lists_infected_nodes.append(cur_infected_ids) 
        lists_infected_dicts.append(cur_infected_dict)
        
    return lists_infected_nodes, lists_infected_dicts



In [10]:
# Thư viện pickle trong Python được sử dụng để thực hiện quá trình "serialization"
# (ghi đối tượng Python thành một chuỗi byte) và "deserialization" (phục hồi đối tượng từ một chuỗi byte).
import pickle

In [18]:
""" Giá trị 20000 cho tham số runs chỉ định rằng chúng ta muốn tạo ra 20000 đợt bùng phát nhiễm.
Giá trị 0.005 cho tham số p_initial_infect chỉ định xác suất ban đầu để một nút bị nhiễm trong mỗi đợt bùng phát. 
Trong trường hợp này, xác suất này được đặt là 0.005, tức là khoảng 0.5%.
Giá trị 0.3 cho tham số p_infect chỉ định xác suất lây nhiễm giữa các nút láng giềng.
Trong trường hợp này, xác suất này được đặt là 0.3, tức là khoảng 30%."""

live_edges_20k_point005_point3, live_edges_20k_point005_point3_steps = createLiveEdges_combined(20000, 0.005, 0.3)

Creating Live Edge #0
Creating Live Edge #2500
Creating Live Edge #5000
Creating Live Edge #7500
Creating Live Edge #10000
Creating Live Edge #12500
Creating Live Edge #15000
Creating Live Edge #17500


In [19]:
pickle.dump(live_edges_20k_point005_point3, open("live_edges_20k_point005_point3", "wb") )
pickle.dump(live_edges_20k_point005_point3_steps, open("live_edges_20k_point005_point3_steps", "wb") )


In [12]:
live_edges_20k_point001_point3, live_edges_20k_point001_point3_steps = createLiveEdges_combined(20000, 0.001, 0.3)

Creating Live Edge #0
Creating Live Edge #2500
Creating Live Edge #5000
Creating Live Edge #7500
Creating Live Edge #10000
Creating Live Edge #12500
Creating Live Edge #15000
Creating Live Edge #17500


In [13]:
pickle.dump(live_edges_20k_point001_point3, open("live_edges_20k_point001_point3", "wb") )
pickle.dump(live_edges_20k_point001_point3_steps, open("live_edges_20k_point001_point3_steps", "wb") )

In [14]:
live_edges_20k_point001_point5, live_edges_20k_point001_point5_steps = createLiveEdges_combined(20000, 0.001, 0.5)

Creating Live Edge #0
Creating Live Edge #2500
Creating Live Edge #5000
Creating Live Edge #7500
Creating Live Edge #10000
Creating Live Edge #12500
Creating Live Edge #15000
Creating Live Edge #17500


In [15]:
pickle.dump(live_edges_20k_point001_point5, open("live_edges_20k_point001_point5", "wb") )
pickle.dump(live_edges_20k_point001_point5_steps, open("live_edges_20k_point001_point5_steps", "wb") )

In [16]:
live_edges_20k_point005_point5, live_edges_20k_point005_point5_steps = createLiveEdges_combined(20000, 0.005, 0.5)

Creating Live Edge #0
Creating Live Edge #2500
Creating Live Edge #5000
Creating Live Edge #7500
Creating Live Edge #10000
Creating Live Edge #12500
Creating Live Edge #15000
Creating Live Edge #17500


In [17]:
pickle.dump(live_edges_20k_point005_point5, open("live_edges_20k_point005_point5", "wb") )
pickle.dump(live_edges_20k_point005_point5_steps, open("live_edges_20k_point005_point5_steps", "wb") )